In [10]:
!pip install pymorphy2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 71.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=9acebebb71f10fd74845d8299c7893690dcd112e4c8079decc335bfc8e715f89
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [11]:
from bs4 import BeautifulSoup 
import requests
import urllib.request
import re
import numpy as np
import pymorphy2
from collections import Counter
import nltk 
nltk.download("punkt")

In [1]:
MALE = 'male'
FEMALE = 'female'
BOTH = 'both'
UNKNOWN = 'unknown' 

In [2]:
MALE_WORDS = set(['guy','spokesman','chairman',"men's",'men','him',"he's",'his',
'boy','boyfriend','boyfriends','boys','brother','brothers','dad',
'dads','dude','father','fathers','fiance','gentleman','gentlemen',
'god','grandfather','grandpa','grandson','groom','he','himself',
'husband','husbands','king','male','man','mr','nephew','nephews',
'priest','prince','son','sons','uncle','uncles','waiter','widower',
'widowers'])

In [3]:
FEMALE_WORDS = set(['heroine','spokeswoman','chairwoman',"women's",'actress','women',
"she's",'her','aunt','aunts','bride','daughter','daughters','female', 'fiancee','girl','girlfriend','girlfriends','girls','goddess',
'granddaughter','grandma','grandmother','herself','ladies',
'lady','mom','moms','mother','mothers','mrs','ms','niece','nieces',
'priestess','princess','queens','she','sister','sisters','waitress',
'widow','widows','wife','wives','woman'])

In [4]:
def genderize(words):

  mwlen = len(MALE_WORDS.intersection(words))
  fwlen = len(FEMALE_WORDS.intersection(words))

  if mwlen > 0 and fwlen == 0:
    return MALE
  elif fwlen > 0 and mwlen == 0:
    return FEMALE
  elif mwlen > 0 and fwlen > 0:
    return BOTH
  else:
    return UNKNOWN

In [6]:
def count_gender(sentences):

  sents = Counter()
  words = Counter()

  for sentence in sentences:
    gender = genderize(sentence)
    sents[gender] += 1
    words[gender] += len(sentence)

  return sents, words

In [38]:
def parse_gender(d1):

  sentences = [
      [word.lower() for word in nltk.word_tokenize(sentence)]
      for sentence in nltk.sent_tokenize(d1)
  ]

  sents, words = count_gender(sentences)
  total = sum(words.values())

  for gender, count in words.items():
    pcent = (count / total) * 100
    nsents = sents[gender]

    print(
        "{} {} ({} sentences)".format(pcent, gender, nsents)
    )

In [12]:
morph = pymorphy2.MorphAnalyzer()

In [13]:
def get_html_statistic(url):

  html = urllib.request.urlopen(url)
  htmlParse = BeautifulSoup(html, 'html.parser')

  n = 0
  d = dict()

  d = htmlParse.get_text('\n', strip = 'True')

  d = re.sub(r'[;,\s]',' ', d)
  # d.get_text()

  # for line in htmlParse.find_all("p"):
  #   # print(line)
  #   for el in line.get_text():
  #     n += 1
  #     d[el] = d.get(el,0) + 1
    # .split(r'\W+', line.get_text().lower()):
    #   # print (el)
    #   n += 1
    #   d[el] = d.get(el,0)+1
  
      # print (el)
      # re.split('\n+',line.get_text().lower())
      # if len(el) > 2:
      #   n += 1
      #   norm_el = morph.parse(el)[0].normal_form
      #   d[norm_el] = d.get(norm_el,0) + 1
      

  return d

In [57]:
d1 = get_html_statistic('https://en.wikipedia.org/wiki/Femininity')
d1

'Femininity - Wikipedia Jump to content Toggle sidebar Search Create account Log in Personal tools Create account Log in Pages for logged out editors learn more Contributions Talk Navigation Main page Contents Current events Random article About Wikipedia Contact us Donate Contribute Help Learn to edit Community portal Recent changes Upload file Tools What links here Related changes Upload file Special pages Permanent link Page information Cite this page Wikidata item Print/export Download as PDF Printable version In other projects Wikimedia Commons Wikiquote Languages On this Wikipedia the language links are at the top of the page across from the article title. Go to top . Contents move to sidebar hide (Top) 1 Overview and history 2 Behavior and personality 3 Clothing and appearance Toggle Clothing and appearance subsection 3.1 In history 3.2 Body alteration 4 Traditional roles Toggle Traditional roles subsection 4.1 Explanations for occupational imbalance 4.1.1 Role congruity theory 

In [34]:
# def parse_gender(url):

#   html = urllib.request.urlopen(url)
#   htmlParse = BeautifulSoup(html, 'html.parser')

#   w = htmlParse.get_text('\n', strip=True)
#   w = re.sub(r'[;,\s]',' ', w)


#   sentences = [
#       [word.lower for word in nltk.word_tokenize(sentence)]
#       for sentence in nltk.sent_tokenize(url)
#   ]

#   sents, words = count_gender(sentences)
#   total = sum(words.values())

#   for gender, count in words.items():
#     pcent = (count / total) * 100
#     nsents = sents[gender]

#     print(
#         "{3.3f}% {} ({} sentences)".format(pcent, gender, nsents)
#     )

In [58]:
parse_gender(d1)

49.85584843492587 unknown (537 sentences)
7.567957166392092 male (24 sentences)
22.487644151565075 female (76 sentences)
20.088550247116967 both (26 sentences)
